In [ ]:
pip install transformers

In [ ]:
from transformers import pipeline

In [ ]:
sentimental_analisis = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
while True:
  sentanse=input("enter your text,'quit' to exit: ")
  if sentanse=='quit':
    break
  print(sentimental_analisis(sentanse))


enter your text,'quit' to exit: quit


sentimental analysis code

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping , ReduceLROnPlateau
from nltk.stem import WordNetLemmatizer , PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras import Sequential
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score , precision_score , recall_score

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Example sequences (e.g., tokenized sentences)
sequences = [
    [1, 2, 3],
    [4, 5, 6, 7, 8],
    [9]
]

# Pad sequences
padded_sequences = pad_sequences(sequences, padding='post', maxlen=5)

print(padded_sequences)


[[1 2 3 0 0]
 [4 5 6 7 8]
 [9 0 0 0 0]]


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Sample text data
texts = [
    "I love machine learning.",
    "Deep learning is a subset of machine learning.",
    "Natural language processing is fun!"
]

# Create and fit the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Tokenize the texts
sequences = tokenizer.texts_to_sequences(texts)

# Get word index
word_index = tokenizer.word_index

# Convert sequences back to text
texts_reversed = tokenizer.sequences_to_texts(sequences)

print("Sequences:", sequences)
print("Word Index:", word_index)
print("Reversed Texts:", texts_reversed)


Sequences: [[4, 5, 2, 1], [6, 1, 3, 7, 8, 9, 2, 1], [10, 11, 12, 3, 13]]
Word Index: {'learning': 1, 'machine': 2, 'is': 3, 'i': 4, 'love': 5, 'deep': 6, 'a': 7, 'subset': 8, 'of': 9, 'natural': 10, 'language': 11, 'processing': 12, 'fun': 13}
Reversed Texts: ['i love machine learning', 'deep learning is a subset of machine learning', 'natural language processing is fun']


In [ ]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("words")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
df = pd.read_csv("Corona_NLP_test.csv", encoding="utf-8")

df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       3798 non-null   int64 
 1   ScreenName     3798 non-null   int64 
 2   Location       2964 non-null   object
 3   TweetAt        3798 non-null   object
 4   OriginalTweet  3798 non-null   object
 5   Sentiment      3798 non-null   object
dtypes: int64(2), object(4)
memory usage: 178.2+ KB


In [ ]:
df.describe()

,UserName,ScreenName
count,3798.000000,3798.000000
mean,1899.500000,46851.500000
std,1096.532489,1096.532489
min,1.000000,44953.000000
25%,950.250000,45902.250000
50%,1899.500000,46851.500000
75%,2848.750000,47800.750000
max,3798.000000,48750.000000


In [ ]:
df.isnull().sum()

,0
UserName,0
ScreenName,0
Location,834
TweetAt,0
OriginalTweet,0
Sentiment,0


In [ ]:
df.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [ ]:
df["Sentiment"].value_counts()

,count
Sentiment,
Negative,1041
Positive,947
Neutral,619
Extremely Positive,599
Extremely Negative,592


In [ ]:
df["OriginalTweet"]

,OriginalTweet
0,TRENDING: New Yorkers encounter empty supermar...
1,When I couldn't find hand sanitizer at Fred Me...
2,Find out how you can protect yourself and love...
3,#Panic buying hits #NewYork City as anxious sh...
4,#toiletpaper #dunnypaper #coronavirus #coronav...
...,...
3793,Meanwhile In A Supermarket in Israel -- People...
3794,Did you panic buy a lot of non-perishable item...
3795,Asst Prof of Economics @cconces was on @NBCPhi...
3796,Gov need to do somethings instead of biar je r...


In [ ]:
df["OriginalTweet"][1]

"When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how  #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY"

In [ ]:
df["OriginalTweet"].fillna("unknown" , inplace=True)

df["OriginalTweet"].replace("" , inplace=True)


<ipython-input-20-4b25fb31cae5>:3: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df["OriginalTweet"].replace("" , inplace=True)


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def process_OriginalTweet(OriginalTweet):

    ## transform OriginalTweet into lowercase
    OriginalTweet = OriginalTweet.lower()

    ## Remove Punctuations
    OriginalTweet = re.sub(r"[^\w\s]" , "" , OriginalTweet)

    ## Remove Numbers
    OriginalTweet = re.sub(r"\d+" , "" , OriginalTweet)

    ## split OriginalTweet into Tokens
    tokens = word_tokenize(OriginalTweet)

    ## Remove StopWords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    stemmer = PorterStemmer()
    stemme_tokens = [stemmer.stem(token) for token in tokens]

    ## Lemmatizer
    lemmatizer = WordNetLemmatizer()
    lemmatize_tokens = [lemmatizer.lemmatize(token) for token in stemme_tokens]

    return " ".join(lemmatize_tokens)

In [ ]:
df["processed_OriginalTweet"] = df["OriginalTweet"].apply(process_OriginalTweet)

In [ ]:
df["processed_OriginalTweet"]

,processed_OriginalTweet
0,trend new yorker encount empti supermarket she...
1,couldnt find hand sanit fred meyer turn amazon...
2,find protect love one coronaviru
3,panic buy hit newyork citi anxiou shopper stoc...
4,toiletpap dunnypap coronaviru coronavirusaustr...
...,...
3793,meanwhil supermarket israel peopl danc sing to...
3794,panic buy lot nonperish item echo need food do...
3795,asst prof econom cconc nbcphiladelphia talk re...
3796,gov need someth instead biar je rakyat assum l...


In [ ]:
df["processed_OriginalTweet"][1]

'couldnt find hand sanit fred meyer turn amazon pack purellcheck coronaviru concern drive price httpstcoygbipbflmi'

In [ ]:
df["processed_OriginalTweet"][0]

'trend new yorker encount empti supermarket shelv pictur wegman brooklyn soldout onlin grocer foodkick maxdeliveri coronavirusfear shopper stock httpstcogrpcrlwh httpstcoivmkmsqdt'

In [ ]:
df["Sentiment"].unique()

array(['Extremely Negative', 'Positive', 'Extremely Positive', 'Negative',
       'Neutral'], dtype=object)

In [ ]:
Sentiment = {"Extremely Negative" :0,
            "Positive":1,
            "Extremely Positive" :2,
            "Negative":3,
            "Neutral":4}

df["Sentiment"] = df["Sentiment"].map(Sentiment)

In [ ]:
import pandas as pd

# Example DataFrame
df = pd.DataFrame({
    'Sentiment': ['positive', 'negative', 'neutral', 'positive']
})

# Dictionary for mapping
sentiment_mapping = {
    'positive': 1,
    'negative': -1,
    'neutral': 0
}

# Apply the mapping
df['Sentiment'] = df['Sentiment'].map(sentiment_mapping)

print(df)

   Sentiment
0          1
1         -1
2          0
3          1


In [ ]:
df["Sentiment"]
df
data = df.drop(["UserName","ScreenName","Location","TweetAt"],axis=1)
data
X = df["processed_OriginalTweet"]

y = df["Sentiment"]
x_train , x_test ,y_train ,y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True)
print("x_train shape" , x_train.shape)
print("x_test shape" , x_test.shape)
print("y_train shape" , y_train.shape)
print("y_test shape" , y_test.shape)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

train_sequence = tokenizer.texts_to_sequences(x_train)
test_sequence = tokenizer.texts_to_sequences(x_test)

max_len = 100

train_sequences = pad_sequences(train_sequence,
                               maxlen=max_len,
                               padding="post",
                               truncating="post")

test_sequences = pad_sequences(test_sequence,
                              maxlen=max_len,
                              padding="post",
                              truncating="post")

print("train_sequences shape" , train_sequences.shape)
print("test_sequences shape" , test_sequences.shape)

KeyError: "['UserName', 'ScreenName', 'Location', 'TweetAt'] not found in axis"

In [ ]:
## Sentiment Analysis

from textblob import TextBlob

def sentiment_analysis(processed_OriginalTweet):
    blob = TextBlob(processed_OriginalTweet)
    classified = blob.sentiment.polarity

    if classified <= 0:
        return "Extremely Negative"
    elif 0 < classified < 2:
        return "Positive"
    elif 1 < classified < 3:
        return "Extremely positive"
    elif 2 < classified < 4:
        return "Negative"
    else:
        return "Neutral"


test_texts = [
    "I absolutely loved this!",
    "This is so bad.",
    "It's okay, not great but not terrible either.",
    "This is a fantastic movie!",
    "The worst experience ever."
]

for text in test_texts:
    sentiment = sentiment_analysis(text)
    print(f"Text: {text}\nSentiment: {sentiment}\n")

In [ ]:
def a (n):
  return n+n



w =[1,2,3,4,5]


x = map(a,w)

print(list(x))

[2, 4, 6, 8, 10]
